In [1]:
import numpy as np
import cv2
import tensorflow as tf
import keras
from tensorflow.python.distribute.mirrored_strategy import MirroredStrategy

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
#from keras.utils import to_categorical
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import (ConvLSTM2D, BatchNormalization, Convolution3D, Convolution2D, Conv2D, Dropout,
                          TimeDistributed, MaxPooling2D, MaxPooling3D, UpSampling2D, UpSampling3D, Input,  AveragePooling3D)

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import os
import gc

2024-01-09 14:20:29.344367: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_enable_xla_devices'
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ['CUDA_VISIBLE_DEVICES'] = "2"
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [3]:
import tensorflow.keras.backend as K

In [4]:
data = np.load("/home/sullah/PycharmProjects/EDSR/Alphorn_Project/dataset_after_augmentation_256x256/del_data_after_augmentation_256x256.npy", mmap_mode='r+')
print(data.shape)

(1900, 32, 256, 256, 1)


In [5]:
# data1 = data[:250]
# data2 = data[250:500]
# data3 = data[500:750]
# data4 = data[750:1000]
# data5 = data[1000:1250]
# data6 = data[1250:1500]
# data7 = data[1500:1750]
data8 = data[1750:]


# print(data1.shape)
# print(data2.shape)
# print(data3.shape)
# print(data4.shape)
# print(data5.shape)
# print(data6.shape)
# print(data7.shape)
print(data8.shape)

#Just run the code for each datasize one by one, as for data1 first then for data2 and so on.

(150, 32, 256, 256, 1)


In [6]:
# load the saved model
model_encoder = tf.keras.models.load_model('ConvLSTM_with_Pooling_with_MSE_autoencoder_decoder-encoder_fully_ConvLSTM_mse.h5', compile = False)
model_decoder = tf.keras.models.load_model('ConvLSTM_with_Pooling_with_MSE_autoencoder_decoder-decoder_fully_ConvLSTM_mse.h5', compile = False)

2024-01-09 14:20:31.894216: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-01-09 14:20:31.895451: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-01-09 14:20:31.963477: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:86:00.0 name: Tesla V100S-PCIE-32GB computeCapability: 7.0
coreClock: 1.597GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 1.03TiB/s
2024-01-09 14:20:31.963541: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-01-09 14:20:31.965767: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-01-09 14:20:31.965844: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2

In [7]:
model_encoder.summary()
model_decoder.summary()

Model: "encoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256, 256, 1)] 0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 32, 256, 256, 8)   2624      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 32, 128, 128, 8)   0         
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 128, 128, 8)   32        
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 32, 128, 128, 32)  46208     
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 32, 64, 64, 32)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 32, 64, 64, 32)    128 

In [8]:
# predictions1 = model_encoder.predict(data1, batch_size=1)
# predictions2 = model_encoder.predict(data2, batch_size=1)
# predictions3 = model_encoder.predict(data3, batch_size=1)
# predictions4 = model_encoder.predict(data4, batch_size=1)
# predictions5 = model_encoder.predict(data5, batch_size=1)
# predictions6 = model_encoder.predict(data6, batch_size=1)
# predictions7 = model_encoder.predict(data7, batch_size=1)
predictions8 = model_encoder.predict(data8, batch_size=1)

2024-01-09 14:20:37.338744: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-01-09 14:20:37.357351: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2100000000 Hz
2024-01-09 14:20:37.961304: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:592] layout failed: Invalid argument: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/Tanh_1' -> 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/mul_5', 'Func/encoder/conv_lst_m2d_2/while/body/_97/input/_240' -> 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/mul_2', 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/convolution_7' -> 'encoder/conv_lst_m2d_2/while/body/_97/encoder/conv_lst_m2d_2/while/add_6', 'encoder/conv_lst_m2d_1/while/body/_49/encoder/conv_lst_m2d_1/while/mul_2' 

In [9]:
print(predictions8.shape)

(150, 32, 32, 32, 256)


In [10]:
np.save('encoder_predictions8', predictions8)